# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
# !pip install requests

In [ ]:
# import the requests library (1 line)

import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)

root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)

status_url = root_url + "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)

if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [ ]:
# import requests
import requests
# Set the countries_url variable (1 line)

countries_url = "https://country-leaders.onrender.com/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)

countries = req.json()

# display the request's status code and the countries variable (1 line)

print(f"Status code: {req.status_code}")
print(countries)

Status code: 403
{'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [ ]:
# Set the cookie_url variable (1 line)

cookie_url = "https://country-leaders.onrender.com/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)

response = requests.get(cookie_url)

cookies = response.cookies



Try to query the countries endpoint using the cookie, save the output and print it.

In [ ]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

countries = requests.get('https://country-leaders.onrender.com/countries', cookies = cookies).json()

# display the countries variable (1 line)

print(countries)

['ma', 'ru', 'be', 'fr', 'us']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [ ]:
# Set the leaders_url variable (1 line)

leaders_url = 'https://country-leaders.onrender.com/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

leaders = requests.get(leaders_url).json()

# leaders = {}
# for element in countries:
#     leaders[element] = requests.get(leaders_url, cookies=cookies, params={"country": element}).json()


# display the leaders variable (1 line)

print(leaders)

{'message': 'The cookie is missing'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)

leaders_url = 'https://country-leaders.onrender.com/leaders'

# assign the output to the leaders variable (1 line)

leaders = requests.get(leaders_url, cookies = cookies, params = {'country': countries[0]}).json()

print(leaders)

[{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines

leaders_per_country = {}

for country in countries:
    response = requests.get(leaders_url, cookies = cookies, params = {'country': country})
    leaders_per_country[country] = response.json()


# Display the leaders_per_country dictionary
print(leaders_per_country)


{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

In [ ]:
# or 1 line

leaders_per_country = {country: requests.get(leaders_url, cookies = cookies, params = {'country': country}).json() for country in countries}

print(leaders_per_country)



{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
# < 15 lines

def get_leaders():
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    
    cookies = requests.get(cookie_url).cookies
    
    countries = requests.get(countries_url, cookies = cookies).json()
    leaders_per_country = {country: requests.get(leaders_url, cookies = cookies, params = {'country': country}).json() for country in countries}
     
    for country, leaders in leaders_per_country.items():
        for leader in leaders:
            print(f"Country: {country}, Leader: {leader}")
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines

leaders_per_country = get_leaders()

print(leaders_per_country)


Country: ma, Leader: {'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}
Country: ma, Leader: {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}
Country: ma, Leader: {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
# 3 lines
leaders_per_country = get_leaders()

leader_url = leaders_per_country['be'][1]['wikipedia_url']

response = requests.get(leader_url)

print(response.text)


Country: ma, Leader: {'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}
Country: ma, Leader: {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}
Country: ma, Leader: {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [he're](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines

from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')

text_context = soup.get_text()

print(text_context)

prettified_html = soup.prettify()

print(prettified_html)






Yves Leterme - Wikipedia



































Naar inhoud springen







Hoofdmenu





Hoofdmenu
naar zijbalk verplaatsen
verbergen



		Navigatie
	


HoofdpaginaVind een artikelVandaagEtalageCategorieënRecente wijzigingenNieuwe artikelenWillekeurige pagina





		Informatie
	


GebruikersportaalSnelcursusHulp en contactDoneren



















Zoeken











Zoeken






















Uiterlijk
















Account aanmaken

Aanmelden








Persoonlijke hulpmiddelen





 Account aanmaken Aanmelden





		Pagina's voor uitgelogde redacteuren meer lezen



BijdragenOverleg




























Inhoud
naar zijbalk verplaatsen
verbergen




Top





1
Levensloop




Toggle Levensloop subsection





1.1
Jeugd








1.2
Politieke loopbaan








1.3
Vlaams minister-president








1.4
Formateur en eerste minister








1.5
Minister van Buitenlandse Zaken en Buitenlandse Handel








1.6
Regering-Leterme II








1.7
Na de Wetstraat






1

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines
paragraphs = soup.find_all('p')

print(paragraphs)

[<p><b>Yves Camille Désiré Leterme</b> (<a href="/wiki/Wervik" title="Wervik">Wervik</a>, <a href="/wiki/6_oktober" title="6 oktober">6 oktober</a> <a href="/wiki/1960" title="1960">1960</a>) is een <a class="mw-redirect" href="/wiki/Belgi%C3%AB_(hoofdbetekenis)" title="België (hoofdbetekenis)">Belgisch</a> <a href="/wiki/Politicus" title="Politicus">politicus</a>. Hij was onder meer <a href="/wiki/Minister-president_van_Vlaanderen" title="Minister-president van Vlaanderen">Vlaams minister-president</a> en <a href="/wiki/Premier_van_Belgi%C3%AB" title="Premier van België">premier van België</a>. Hij was lang boegbeeld van de <a href="/wiki/Christen-Democratisch_en_Vlaams" title="Christen-Democratisch en Vlaams">CD&amp;V</a> en boegbeeld van het toenmalige kartel <a href="/wiki/Valentijnskartel" title="Valentijnskartel">CD&amp;V/N-VA</a>.
</p>, <p>Hij werd geboren in <a href="/wiki/Wervik" title="Wervik">Wervik</a>, op de grens tussen <a href="/wiki/Frankrijk" title="Frankrijk">Frankrij

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [ ]:
# <10 lines

for i in paragraphs:
    paragraph_text = i.get_text().strip()
    if len(paragraph_text) > 30:
        first_paragraph_text = paragraph_text
        break
        
print(paragraph_text)        
    

Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.


In [ ]:
# <10 lines

for i in paragraphs:
    paragraph_text = i.get_text().strip()
    if paragraph_text:
        first_paragraph_text = paragraph_text
        break
        
print(paragraph_text)        
    

Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

In [ ]:
# 10 lines

def get_first_paragraph(wikipedia_url):
   print(wikipedia_url) # keep this for the rest of the notebook
   
   response = requests.get(wikipedia_url)

   soup = BeautifulSoup(response.text, 'html.parser') 
    
   paragraphs = soup.find_all('p')
    
   first_paragraph = None 

   for i in paragraphs:
    text = i.get_text().strip()
    if text:
        first_paragraph = text
        break
    
   return first_paragraph



In [ ]:
wikipedia_url = 'https://nl.wikipedia.org/wiki/Wilfried_Martens'

get_first_paragraph(wikipedia_url)

https://nl.wikipedia.org/wiki/Wilfried_Martens


'Wilfried Achiel Emma Martens uitspraakⓘ (Sleidinge, 19 april 1936 – Lokeren, 9 oktober 2013) was een Vlaams-Belgische politicus voor de CD&V (vroeger CVP) en was van april 1979 tot maart 1992 (met een korte onderbreking) eerste minister van België.'

In [ ]:
# Test: 3 lines

print(get_first_paragraph(wikipedia_url))

https://nl.wikipedia.org/wiki/Wilfried_Martens
Wilfried Achiel Emma Martens uitspraakⓘ (Sleidinge, 19 april 1936 – Lokeren, 9 oktober 2013) was een Vlaams-Belgische politicus voor de CD&V (vroeger CVP) en was van april 1979 tot maart 1992 (met een korte onderbreking) eerste minister van België.


In [ ]:
#  If I want to get a wikipedia url :


def extract_wikipedia_url(leaders_per_country):
    wikipedia_urls = {}
    
    for country, leaders in leaders_per_country.items():
        country_urls = []
        for leader in leaders:
            if 'wikipedia_url' in leader:
                country_urls.append(leader['wikipedia_url'])
        wikipedia_urls[country] = country_urls        
    return wikipedia_urls   
leaders_per_country = get_leaders()
wikipedia_urls = extract_wikipedia_url(leaders_per_country)

for country, urls in wikipedia_urls.items():
    print(f"{country}: {urls}")

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re

clean_text = re.sub(r'\[\d+\]', '', get_first_paragraph(wikipedia_url))
clean_text = re.sub(r'\([^)]*\)', '', clean_text)

print(clean_text.strip())



https://nl.wikipedia.org/wiki/Wilfried_Martens
Wilfried Achiel Emma Martens uitspraakⓘ  was een Vlaams-Belgische politicus voor de CD&V  en was van april 1979 tot maart 1992  eerste minister van België.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines

def get_first_paragraph(wikipedia_url):
   print(wikipedia_url) # keep this for the rest of the notebook
   
   response = requests.get(wikipedia_url)

   soup = BeautifulSoup(response.text, 'html.parser') 
    
   paragraphs = soup.find_all('p')
    
   first_paragraph = "" 

   for paragraph in paragraphs:    
        if paragraph.find("b"):
            first_paragraph += paragraph.get_text()
            break
    
   if first_paragraph:
        clean_text = re.sub(r'\[\d+\]', '', first_paragraph)
        
        clean_text = re.sub(r'\([^)]*\)', '', clean_text)
        
        return clean_text.strip()  
    
   return None 


print(get_first_paragraph(wikipedia_url))

https://en.wikipedia.org/wiki/Herbert_Hoover
Herbert Clark Hoover  was an American politician and humanitarian who served as the 31st president of the United States from 1929 to 1933. A member of the Republican Party, he held office during the onset of the Great Depression. A wealthy mining engineer before his presidency, Hoover led the wartime Commission for Relief in Belgium, served as the director of the U.S. Food Administration, and served as the U.S. Secretary of Commerce.


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:

def get_first_paragraph(wikipedia_url):
   # print(wikipedia_url) # keep this for the rest of the notebook
   
   response = requests.get(wikipedia_url)

   soup = BeautifulSoup(response.text, 'html.parser') 
    
   paragraphs = soup.find_all('p')
    
   first_paragraph = "" 

   for paragraph in paragraphs:    
        if paragraph.find("b"):
            first_paragraph += paragraph.get_text()
            break
    
    
   if first_paragraph:
    clean_text = re.sub(r'\[\d+\]', '', first_paragraph)
        
    clean_text = re.sub(r'\([^)]*\)', '', clean_text)
        
    clean_text = re.sub(r'<.*?>', '', clean_text)
        
    clean_text = clean_text.encode('ascii', 'ignore').decode('utf-8')
        
    clean_text = re.sub(r'\s+', ' ', clean_text)
    return clean_text.strip() 
    
   return None 

print(get_first_paragraph(wikipedia_url))

Herbert Clark Hoover was an American politician and humanitarian who served as the 31st president of the United States from 1929 to 1933. A member of the Republican Party, he held office during the onset of the Great Depression. A wealthy mining engineer before his presidency, Hoover led the wartime Commission for Relief in Belgium, served as the director of the U.S. Food Administration, and served as the U.S. Secretary of Commerce.


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    
    cookies = requests.get(cookie_url).cookies
    
    countries = requests.get(countries_url, cookies=cookies).json()

In [ ]:
# < 20 lines


def get_leader():
    leaders_per_country = {}
    
    for country in countries:
        leaders = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        
        leaders_with_paragraph = []
        
        for leader in leaders:
            if 'wikipedia_url' in leader:
                wikipedia_url = leader['wikipedia_url']
                first_paragraph = get_first_paragraph(wikipedia_url)
                leader['first_paragraph'] = first_paragraph
                leaders_with_paragraph.append(leader)
            else:
                print(f"No wikipeia URL found for for leader: {leader}")
    
        leaders_per_country[country] = leaders_with_paragraph
        
    return leaders_per_country   
    
leaders_per_country = get_leader()

for country, leaders in leaders_per_country.items():
    print(f"Country: {country}")
    for leader in leaders:
        print(f"Leader: {leader['first_name']} {leader['last_name']}")
        print(f"First Paragraph: {leader['first_paragraph']}")
        print("------------------------")


No wikipeia URL found for for leader: message
No wikipeia URL found for for leader: message
Country: ma
Leader: Mohammed None
First Paragraph: 1999 9 1420 23 1999 .
------------------------
Leader: Hassan None
First Paragraph: 26 1961 20 1999. 1666 .
------------------------
Leader: Mohammed None
First Paragraph: 22 1346 17 1927 23 1346 18 1927 1957 1957 1961 . . . 16 1955. . : .
------------------------
Leader: Mohammed None
First Paragraph: 1509 - 1517.
------------------------
Country: ru
Leader: Vladimir Putin
First Paragraph: . , ; 7 2012. 7 2000 7 2008, 19992000 20082012 . , , 1999 2000. 2017 , II.
------------------------
Leader: Dmitry Medvedev
First Paragraph: :
------------------------
Leader: Boris Yeltsin
First Paragraph: , , , , 23 2007, , ) , , [b]; 1991 1992 . 1992 .
------------------------
Country: be
Leader: Guy Verhofstadt
First Paragraph: Guy Maurice Marie-Louise Verhofstadt is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten . Hij was premier van

In [ ]:
# Check the output of your function (2 lines)
leaders_per_country = get_leader()

for country, leaders in leaders_per_country.items():
    print(f"Country: {country}")
    for leader in leaders:
        print(f"Leader: {leader['first_name']} {leader['last_name']}")
        print(f"First Paragraph: {leader['first_paragraph']}")
        print("------------------------")


No wikipeia URL found for for leader: message
No wikipeia URL found for for leader: message
No wikipeia URL found for for leader: message
No wikipeia URL found for for leader: message
No wikipeia URL found for for leader: message
Country: ma
Country: ru
Country: be
Country: fr
Country: us


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines

def get_leader():
    def get_cookies():
        return requests.get('https://country-leaders.onrender.com/cookie').cookies
    
    cookies = get_cookies()
    
    leaders_per_country = {}
    
    for country in countries:
        try:
            leaders = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        except requests.exceptions.RequestException:
            cookies = get_cookies()
            leaders = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
            
        leaders_with_paragraph = []
            
        for leader in leaders:
             if 'wikipedia_url' in leader:
                wikipedia_url = leader['wikipedia_url']
                first_paragraph = get_first_paragraph(wikipedia_url)
                leader['first_paragraph'] = first_paragraph
                leaders_with_paragraph.append(leader)
             else:
                print(f"No wikipedia URL found for leader: {leader}")
                
        leaders_per_country[country] = leaders_with_paragraph
    
    return leaders_per_country   
    
leaders_per_country = get_leader()

for country, leaders in leaders_per_country.items():
    print(f"Country: {country}")
    for leader in leaders:
        print(f"Leader: {leader['first_name']} {leader['last_name']}")
        print(f"First Paragraph: {leader['first_paragraph']}")
        print("------------------------")    


Check the output of your function again.

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines

def get_first_paragraph(wikipedia_url, session):
    try:
        response = session.get(wikipedia_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        first_paragraph = None
        for paragraph in soup.find_all('p'):
            text = paragraph.get_text().strip()
            if text:
                first_paragraph = text
                break
        
        if first_paragraph:
            return first_paragraph
        else:
            print(f"No meaningful paragraph found on {wikipedia_url}")
            return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Wikipedia page: {e}")
        return None
session = requests.Session()    
first_paragraph = get_first_paragraph('https://en.wikipedia.org/wiki/Albert_Einstein', session)
print(f"First Paragraph: {first_paragraph}")

First Paragraph: Albert Einstein (/ˈaɪnstaɪn/ EYEN-styne;[4] German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ⓘ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century.[1][5] His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation".[6] He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect",[7] a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science.[8][9]


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_first_paragraph(wikipedia_url, session):
    try:
        response = session.get(wikipedia_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        first_paragraph = soup.find('p').get_text()  
        return first_paragraph
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Wikipedia page: {e}")
        return None

def get_leaders(session):
    leaders_per_country = {}
    cookies = get_cookies()  
    
    for country in countries:
        try:
            leaders = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        except requests.exceptions.RequestException:
            cookies = get_cookies()
            leaders = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        
        leaders_with_paragraph = []
        
        for leader in leaders:
            if 'wikipedia_url' in leader:
                wikipedia_url = leader['wikipedia_url']
                first_paragraph = get_first_paragraph(wikipedia_url, session)
                leader['first_paragraph'] = first_paragraph
                leaders_with_paragraph.append(leader)
            else:
                print(f"No Wikipedia URL found for leader: {leader}")
        
        leaders_per_country[country] = leaders_with_paragraph
    
    return leaders_per_country


leaders_per_country = get_leaders(session)

for country, leaders in leaders_per_country.items():
    print(f"Country: {country}")
    for leader in leaders:
        print(f"Leader: {leader['first_name']} {leader['last_name']}")
        print(f"First Paragraph: {leader['first_paragraph']}")
        print("------------------------")


Country: ma
Leader: Mohammed None
First Paragraph: مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.

------------------------
Leader: Hassan None
First Paragraph: الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)،[12] ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 20 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.

------------------------
Leader: Mohammed None
First Paragraph: المستوىمنظورة

------------------------
Leader: Mohammed None
First P

In [ ]:
# <25 lines
import requests

def get_leader(countries, leaders_url, session):
    leaders_per_country = {}
    
    for country in countries:
        try:
            leaders = requests.get(leaders_url, params={'country': country}, cookies=session.cookies).json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching leaders for {country}: {e}")
            continue
        
        leaders_with_paragraph = []
        
        for leader in leaders:
            if 'wikipedia_url' in leader:
                wikipedia_url = leader['wikipedia_url']
                first_paragraph = get_first_paragraph(wikipedia_url, session)
                leader['first_paragraph'] = first_paragraph
                leaders_with_paragraph.append(leader)
            else:
                print(f"No wikipedia URL found for leader: {leader}")
                
        leaders_per_country[country] = leaders_with_paragraph
    
    return leaders_per_country


Test your new functions.



In [ ]:
session = requests.Session()

leaders_per_country = get_leaders(session)

for country, leaders in leaders_per_country.items():
    print(f"Country: {country}")
    for leader in leaders:
        print(f"Leader: {leader['first_name']} {leader['last_name']}")
        print(f"First Paragraph: {leader['first_paragraph']}")
        print("------------------------")

Country: ma
Leader: Mohammed None
First Paragraph: مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.

------------------------
Leader: Hassan None
First Paragraph: الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)،[12] ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 20 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.

------------------------
Leader: Mohammed None
First Paragraph: المستوىمنظورة

------------------------
Leader: Mohammed None
First P

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json
with open('leaders.json', 'w') as json_file:
    json.dump(leaders_per_country, json_file, indent=4)
    print("Data saved successfully to leaders.json.")

Data saved successfully to leaders.json.


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open('leaders.json', 'r') as json_file:
    saved_leaders_per_country = json.load(json_file)

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
def save(leaders_per_country):
    with open('leaders.json', 'w') as json_file:
        json.dump(leaders_per_country, json_file, indent=4)

In [ ]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!